In [ ]:
import subprocess
import wave
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import os

class AudioProcessor:
    def __init__(self, input_path, temp_output_path='temp_output.wav'):
        """
        Initialize the AudioProcessor class with the input path.

        Parameters:
        input_path (str): The file path of the input M4A audio file.
        temp_output_path (str): The file path for the temporary WAV file. Defaults to 'temp_output.wav'.
        """
        self.input_path = input_path
        self.wav_file_path = temp_output_path
        self.samples = None
        self.sample_rate = None

    def convert_m4a_to_wav(self):
        """
        Convert an M4A file to WAV format using ffmpeg.
        
        Raises:
        subprocess.CalledProcessError: If the ffmpeg conversion fails.
        """
        try:
            subprocess.run(['ffmpeg', '-i', self.input_path, self.wav_file_path], check=True)
        except subprocess.CalledProcessError as e:
            raise RuntimeError(f"Error converting {self.input_path} to WAV: {e}")

    def read_wav(self):
        """
        Read a WAV file and load audio samples and sample rate.
        
        Raises:
        wave.Error: If there is an issue reading the WAV file.
        """
        try:
            with wave.open(self.wav_file_path, 'rb') as wav_file:
                self.sample_rate = wav_file.getframerate()
                n_frames = wav_file.getnframes()
                n_channels = wav_file.getnchannels()
                frames = wav_file.readframes(n_frames)
                
                # Convert the byte data to numpy array
                self.samples = np.frombuffer(frames, dtype=np.int16)
                
                if n_channels == 2:
                    self.samples = self.samples.reshape((-1, 2))
                    self.samples = self.samples.mean(axis=1)  # Convert to mono by averaging channels
        except wave.Error as e:
            raise RuntimeError(f"Error reading WAV file: {e}")

    def compute_spectrum(self, ceiling=None):
        """
        Compute the Fourier Transform of the audio samples and return frequency and amplitude.

        Parameters:
        ceiling (float, optional): The ceiling frequency to filter the output. Defaults to None.
        
        Returns:
        tuple: A tuple containing the frequency array (numpy array) and the amplitude array (numpy array).
        """
        n = len(self.samples)
        T = 1.0 / self.sample_rate
        yf = np.fft.fft(self.samples)
        xf = np.fft.fftfreq(n, T)[:n//2]
        amplitude = 2.0 / n * np.abs(yf[:n//2])
        
        if ceiling is not None:
            mask = xf <= ceiling
            xf = xf[mask]
            amplitude = amplitude[mask]
        
        return xf, amplitude

    def get_decibel_values(self):
        """
        Compute and return the decibel values of the audio file.
        
        Returns:
        tuple: A tuple containing the frequency array (numpy array) and the decibel array (numpy array).
        """
        self.convert_m4a_to_wav()
        self.read_wav()
        xf, amplitude = self.compute_spectrum()
        decibel_values = 20 * np.log10(amplitude)
        return xf, decibel_values

    def plot_spectrum(self, xf, amplitude, title, ylabel, log_scale=False):
        """
        Plot the frequency spectrum using matplotlib.
        
        Parameters:
        xf (numpy array): The frequency array.
        amplitude (numpy array): The amplitude array.
        title (str): The title of the plot.
        ylabel (str): The label for the y-axis.
        """
        plt.figure(figsize=(12, 6))
        plt.plot(xf, amplitude)
        plt.title(title)
        plt.xlabel('Frequency (Hz)')
        plt.ylabel(ylabel)
        if log_scale:
            plt.xscale('log')
        plt.grid()
        plt.show()
    
    def plot_spectrum_plotly(self, xf, amplitude, title, ylabel, log_scale=False):
        """
        Plot the frequency spectrum using Plotly.

        Parameters:
        xf (numpy array): The frequency array.
        amplitude (numpy array): The amplitude array.
        title (str): The title of the plot.
        ylabel (str): The label for the y-axis.
        log_scale (bool): Whether to use a log scale for the x-axis.

        Returns:
        plotly.graph_objects.Figure: The Plotly figure object.
        """
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=xf, y=amplitude, mode='lines'))
        fig.update_layout(
            title=title,
            xaxis_title='Frequency (Hz)',
            yaxis_title=ylabel,
            template='plotly_dark'
        )
        if log_scale:
            fig.update_xaxes(type="log")
        return fig
    
    def export_plotly_html(self, fig, file_path):
        """
        Export a Plotly figure to an HTML file.
        
        Parameters:
        fig (plotly.graph_objects.Figure): The Plotly figure.
        file_path (str): The file path to save the HTML file.
        """
        fig.write_html(file_path)

    def perform_spectral_analysis(self, ceiling_frequency=None):
        """
        Perform spectral analysis on the audio file and generate plots.

        Parameters:
        ceiling_frequency (float, optional): The ceiling frequency to filter the output. Defaults to None.
        """
        try:
            self.convert_m4a_to_wav()
            self.read_wav()
            xf, amplitude = self.compute_spectrum(ceiling=ceiling_frequency)

            # Plot linear amplitude
            self.plot_spectrum(xf, amplitude, 'Frequency Spectrum', 'Amplitude')
            
            fig = self.plot_spectrum_plotly(xf, amplitude, 'Frequency Spectrum', 'Amplitude')
            self.export_plotly_html(fig, 'frequency_spectrum_linear.html')
            fig.show()

            # Plot with log scale
            self.plot_spectrum(xf, amplitude, 'Frequency Spectrum (Log Scale)', 'Amplitude', log_scale=True)

            fig = self.plot_spectrum_plotly(xf, amplitude, 'Frequency Spectrum (Log Scale)', 'Amplitude', log_scale=True)
            self.export_plotly_html(fig, 'frequency_spectrum_log_scale.html')
            fig.show()
        finally:
            # Cleanup temporary files
            if os.path.exists(self.wav_file_path):
                os.remove(self.wav_file_path)

# Usage example
audio_processor = AudioProcessor("C-21231103A_1st.m4a")
audio_processor.perform_spectral_analysis()

# Get decibel values
frequencies, decibel_values = audio_processor.get_decibel_values()
print(frequencies)
print(decibel_values)